In [27]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
import re

In [28]:
load_dotenv()

model = ChatGoogleGenerativeAI(model='gemini-3-flash-preview')

In [29]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str
    score: float

In [30]:
def to_text(content) -> str:
    if isinstance(content, list):
        parts = []
        for part in content:
            if isinstance(part, dict) and "text" in part:
                parts.append(part["text"])
            else:
                parts.append(str(part))
        return "".join(parts)
    return str(content)

In [31]:
def create_outline(state: BlogState) -> BlogState:

    title = state['title']

    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = to_text(model.invoke(prompt).content)

    state['outline'] = outline

    return state

In [32]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = to_text(model.invoke(prompt).content)

    state['content'] = content

    return state

In [33]:
def evaluate_blog(state: BlogState) -> BlogState:

    blog = to_text(state['content'])

    prompt = (
        'Evaluate the following blog on a scale of 1-10 based on content quality, '
        'grammar and engagement. Provide only the score.\n '
        f'{blog}'
    )

    score_raw = model.invoke(prompt).content
    score_text = to_text(score_raw).strip()

    match = re.search(r"\d+(?:\.\d+)?", score_text)
    if not match:
        raise ValueError(f"Unexpected score format: {score_text}")

    state['score'] = float(match.group())

    return state

In [34]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluate_blog', evaluate_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluate_blog')
graph.add_edge('evaluate_blog', END)

workflow = graph.compile()

In [35]:
intial_state = {'title': 'Rise of AI in India'}
final_state = workflow.invoke(intial_state)
print(final_state)

{'title': 'Rise of AI in India', 'outline': 'This detailed blog outline covers the evolution, impact, challenges, and future of Artificial Intelligence in India. It is structured to provide a logical flow for a comprehensive long-form article.\n\n---\n\n# Blog Title Options:\n*   *The Silicon Pulse: How AI is Transforming India’s Digital Landscape*\n*   *From IT Hub to AI Powerhouse: The Rise of Artificial Intelligence in India*\n*   *AI for All: Decoding India’s Unique Path to Artificial Intelligence Mastery*\n\n---\n\n## I. Introduction\n*   **The Hook:** Start with a powerful statistic (e.g., India’s AI market projection or the number of AI startups).\n*   **The Shift:** Briefly explain India’s transition from the world\'s "back-office" (IT services) to an "innovation hub" (AI development).\n*   **Thesis Statement:** AI is no longer a futuristic concept in India; it is the backbone of the country’s mission to become a $5 trillion economy.\n*   **Roadmap:** A quick overview of what t

In [37]:
print(final_state['content'])

# From IT Hub to AI Powerhouse: The Rise of Artificial Intelligence in India

By 2027, India’s Artificial Intelligence (AI) market is projected to reach a staggering **$17 billion**, growing at an annual rate of 25-35%. While these numbers are impressive, they only tell half the story. The real narrative lies in the bustling tech corridors of Bengaluru, the remote farms of Maharashtra, and the government offices in New Delhi.

For decades, India was known as the world’s "back-office," the go-to destination for cost-effective IT services. Today, that narrative is shifting. India is rapidly evolving from a service provider into a global **innovation hub**, where AI is no longer a futuristic concept but the backbone of a mission to become a $5 trillion economy.

In this blog, we will decode India’s unique path to AI mastery—exploring the foundations of this boom, the sectors being revolutionized, the government’s ambitious "AI for All" vision, and the challenges that lie ahead.

---

## I

In [38]:
final_state['outline']

'This detailed blog outline covers the evolution, impact, challenges, and future of Artificial Intelligence in India. It is structured to provide a logical flow for a comprehensive long-form article.\n\n---\n\n# Blog Title Options:\n*   *The Silicon Pulse: How AI is Transforming India’s Digital Landscape*\n*   *From IT Hub to AI Powerhouse: The Rise of Artificial Intelligence in India*\n*   *AI for All: Decoding India’s Unique Path to Artificial Intelligence Mastery*\n\n---\n\n## I. Introduction\n*   **The Hook:** Start with a powerful statistic (e.g., India’s AI market projection or the number of AI startups).\n*   **The Shift:** Briefly explain India’s transition from the world\'s "back-office" (IT services) to an "innovation hub" (AI development).\n*   **Thesis Statement:** AI is no longer a futuristic concept in India; it is the backbone of the country’s mission to become a $5 trillion economy.\n*   **Roadmap:** A quick overview of what the blog will cover (Sectors, Government role

In [39]:
print(final_state['score'])

9.0
